#### IMPORTS

In [23]:
from faker import Faker
import pandas as pd
import random
import re
import csv

fake = Faker()

### Mentors

In [24]:
academic_mentors = []
industry_mentors = []
universities = [
    "University of Dundee",
    "University of Edinburgh",
    "University of Glasgow"
]
uni_domain = {"University of Dundee" :"dundee.ac.uk",
    "University of Edinburgh":"edinburgh.ac.uk",
    "University of Glasgow":"glasgow.ac.uk"}
employers = [
    "Tata Consultancy Services",
    "Amazon Development Centre Scotland",
    "JP Morgan Chase & Co.",
    "Barclays Technology Centre",
    "Skyscanner",
    "Deloitte UK",
    "IBM UK Ltd.",
    "BAE Systems Applied Intelligence",
    "Morgan Stanley",
    "Accenture Scotland"
]

for i in range(20):
    name = fake.name()
    email_name = name.lower().replace(" ",".")
    univerty = random.choice(universities)
    domain = uni_domain[univerty]
    academic_mentors.append({
        "mentor_id" : f'AM{i+1:03d}',
        "name": name,
        "email":f'{email_name}@{domain}',
        "university": univerty
    })
for i in range(20):
    industry_mentors.append({
        "mentor_id" : f'IM{i+1:03d}',
        "name" : fake.name(),
        "email" : fake.company_email(),
        "employer" : random.choice(employers)
    })




### Creating Students Profiles

### Students

In [25]:
#students.csv
courses = [
    "Applied Data Science",
    "Software Engineering Principles",
    "Computational Modelling and Programming",
    "Human-Centred Computing",
    "Cybersecurity and Ethical Hacking",
    "Professional Project Management",
    "Research Methods in Computing",
    "Interactive Media Design",
    "Big Data Analysis and Visualisation",
    "Artificial Intelligence and Machine Learning",
    "Business Intelligence and Strategy",
    "Information Systems Management"
]
students = []
years = [1,2,3,4]

In [26]:
#students
for i in range(100): #no of students 100
   name = fake.name()
   students_email = name.lower().replace(" ",".")
   students.append({
                   "student_id":f"S{i+1:03d}",
                   "name": name,
                   "email":f'{students_email}@gmail.com',
                   "course":random.choice(courses),
                   "current_Year":random.choice(years),
                   "academic_mentors":random.choice(academic_mentors)["mentor_id"],
                   "industry_mentors":random.choice(industry_mentors)["mentor_id"]
   }
   )

### Converting to DataFrames

In [27]:
students_df= pd.DataFrame(students) #Converting the raw data in the form of Dataframe/Tables
academic_mentors_df = pd.DataFrame(academic_mentors)
industry_mentors_df = pd.DataFrame(industry_mentors)

### Exporting the .csv

In [28]:

students_df.to_csv("students.csv", index=False)
print("CSV file saved: students.csv")

academic_mentors_df.to_csv("academnic_mentors.csv",index = False)
industry_mentors_df.to_csv("industry_mentors.csv",index = False)

print("CSV file saved: academic_mentors.csv and industry_mentors.csv ")

CSV file saved: students.csv
CSV file saved: academic_mentors.csv and industry_mentors.csv 


### Assignment

In [29]:
assignment = []

def get_grade(submitted,total):
    if submitted == total:
        return "A1"
    elif submitted == total-1:
        return "B1"
    elif submitted == total-2:
        return "B2"
    elif submitted >=1:
        return "C1"
    else:
        return "Fail"
    
courses_assignment = {
    "Applied Data Science": 3,
    "Software Engineering Principles":5,
    "Computational Modelling and Programming":3,
    "Human-Centred Computing":5,
    "Cybersecurity and Ethical Hacking":4,
    "Professional Project Management":4,
    "Research Methods in Computing":3,
    "Interactive Media Design":4,
    "Big Data Analysis and Visualisation":3,
    "Artificial Intelligence and Machine Learning":4,
    "Business Intelligence and Strategy":5,
    "Information Systems Management":5
}

for i,students in enumerate(students):
    assignment_id = f'AS{i+1:03d}'
    courses = students["course"]
    student_id = students["student_id"]
    total = courses_assignment.get(courses)
    submitted = random.randint(0,total)
    grade = get_grade(submitted,total)
    
    assignment.append({
        "assignment_id": assignment_id,
        "student_id": student_id,
        "total_assignments": total,
        "submitted_assignments": submitted,
        "assignment_grade": grade
    })
    
assignment_df = pd.DataFrame(assignment)
assignment_df.to_csv("assignment.csv",index=False)

### Academic progress


In [30]:
academic_progress = []

for i in range(100):  
    student_id = students_df.iloc[i]["student_id"]
    assignment_row = assignment_df[assignment_df["student_id"] == student_id]
    assignment_grade = assignment_row.iloc[0]["assignment_grade"]
    
    academic_progress.append({
        "academic_progress_id" : f'AP{i+1:03d}',
        "student_id": student_id,
        "percentage": f'{random.randint(40,98)}',
        "assignment": assignment_grade, 
        "attendance": random.randint(60,100),
        "last_update" : fake.date_this_month().strftime("%Y-%m-%d")
    }
    )

academic_progress_df = pd.DataFrame(academic_progress)
academic_progress_df.to_csv("academic_progress.csv",index=False)
print("CSV file saved: academic_progress.csv")

CSV file saved: academic_progress.csv


### Industry Log

In [31]:
industry_progress = []

for i in range(100):
    student_id = students_df.iloc[i]["student_id"]

    industry_progress.append({
         "industry_id": f'IL{i+1:03d}',
         "student_id" : student_id,
         "attendance" : random.randint(60,98),
         "Project"  : random.randint(1,2)
    })
    
industry_progress_df = pd.DataFrame(industry_progress)
industry_progress_df.to_csv("industry_progress.csv",index=False)
    

### Meetings

In [32]:
meetings = []

for i in range(100):
    student_id = students_df.iloc[i]["student_id"]
    mentor_id = random.choice([students_df.iloc[i]["academic_mentors"],
                              students_df.iloc[i]["industry_mentors"]])

    meetings.append({
        "meeting_id" : f'MEET{i:03d}',
        "student_id" : student_id,
        "mentor_id" : mentor_id,
        "date" : fake.date_this_month().strftime("%Y-%m-%d")
    })
    
meetings_df = pd.DataFrame(meetings)
meetings_df.to_csv("meetings.csv",index=False)

### Exception Log

In [33]:
exception = []

exceptions_types= [
    # Attendance Exceptions
    {"code": "EXC001", "reason": "Medical Emergency", "category": "attendance"},
    {"code": "EXC002", "reason": "Family Emergency", "category": "attendance"},
    {"code": "EXC003", "reason": "Corporate Overwork / High Workload", "category": "attendance"},
    {"code": "EXC004", "reason": "Approved Leave (Vacation, Wedding, etc.)", "category": "attendance"},
    {"code": "EXC007", "reason": "Mental Health Leave / Counselling", "category": "attendance"},
    {"code": "EXC010", "reason": "Visa or Legal Paperwork Delays", "category": "attendance"},

    # Assignment Exceptions
    {"code": "EXC001", "reason": "Medical Emergency", "category": "assignment"},
    {"code": "EXC002", "reason": "Family Emergency", "category": "assignment"},
    {"code": "EXC003", "reason": "Corporate Overwork / High Workload", "category": "assignment"},
    {"code": "EXC008", "reason": "Miscommunication or Delay in Assignment Communication", "category": "assignment"},
    {"code": "EXC009", "reason": "Extension Approved by Academic Mentor", "category": "assignment"},

    # Meeting Exceptions
    {"code": "EXC006", "reason": "Mentor Unavailability / Meeting Canceled by Mentor", "category": "meeting"},
    {"code": "EXC001", "reason": "Medical Emergency", "category": "meeting"},
    {"code": "EXC002", "reason": "Family Emergency", "category": "meeting"},
    {"code": "EXC003", "reason": "Corporate Overwork / High Workload", "category": "meeting"},
]



for i in range(30):
    choosen_expetion_reason = random.choice(exceptions_types)
    
    reason = choosen_expetion_reason["reason"]
    category = choosen_expetion_reason["category"]
    
    
    student_id = random.choice(students_df["student_id"].tolist())
    
    
    exception.append({
        "exception_id": f'EX{i+1:03d}',
        "student_id" : student_id,
        "reason": reason,
        "category": category
    })


exception_df = pd.DataFrame(exception)
exception_df.to_csv("exception.csv",index=False)